In [3]:
# modelo de base con 3 bloques vgg para la base cifar1
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [4]:
# cargar bases de entrenamiento y prueba
def load_dataset():
	# cargar bases
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one-hot-encode las etiquetas
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [5]:
# escalar los pixeles
def prep_pixels(train, test):
	# convertir de integers a floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalizae en rango 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# returna iamgenes normalizadas
	return train_norm, test_norm

In [6]:
# definir modelo cnn inspirado en vgg
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compilar modelo
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model


In [7]:
# mostrar las curvas de diagnóstico del aprendizaje
def summarize_diagnostics(history):
	# plot error
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot exactitud
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# guardar el plot en un archivo ... por si lo queremos publicar
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [8]:
# ejecutamos el ambiente de evaluación de modelos
def run_amb_pruebas():
	# cargar el dataset
	trainX, trainY, testX, testY = load_dataset()
	# preparar los pixeles
	trainX, testX = prep_pixels(trainX, testX)
	# definir el modelo
	model = define_model()
	# entrenar el modelo
	history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)
	# evaluar el modelo
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# curvas de aprendizaje
	summarize_diagnostics(history)

In [9]:
# punto de entrada del ambiente de evaluación de modelos
run_amb_pruebas()

Metal device set to: Apple M1


2021-12-03 12:35:40.942164: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-03 12:35:40.942380: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/enrique/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-12-03 12:35:42.061336: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-03 12:35:42.061618: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU freque

> 73.770
